<a href="https://colab.research.google.com/github/Sana-0511/Document_Summarization-/blob/main/NetworkX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the Natural Language Toolkit (nltk) for natural language processing tasks
import nltk
# Import the stopwords corpus from nltk, which contains common words to be ignored in text analysis
from nltk.corpus import stopwords
# Import cosine_distance function from nltk's cluster module to compute cosine similarity between vectors
from nltk.cluster.util import cosine_distance
# Import NumPy, a library for numerical operations, to work with numerical arrays
import numpy as np
# Import NetworkX, a library for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Install the required utility
!apt-get install wget

# Import the necessary libraries
import zipfile
import os

# Provide the Google Drive link
google_drive_link = 'https://drive.google.com/uc?id=1FV5aYhpvl6NFrH7aYzEVg8pmO_yokYoI'

# Specify the output file name
zip_file = 'example.zip'

# Download the zip file
!wget --no-check-certificate $google_drive_link -O $zip_file

# Create a directory to extract the contents
extracted_dir = 'extracted_content'
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# List the files in the extracted directory
extracted_files = os.listdir(extracted_dir)
print("Files extracted:", extracted_files)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
--2024-01-12 09:25:38--  https://drive.google.com/uc?id=1FV5aYhpvl6NFrH7aYzEVg8pmO_yokYoI
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.102, 142.251.2.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1FV5aYhpvl6NFrH7aYzEVg8pmO_yokYoI [following]
--2024-01-12 09:25:38--  https://drive.usercontent.google.com/download?id=1FV5aYhpvl6NFrH7aYzEVg8pmO_yokYoI
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.2.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.2.132|:443... connected.
HTTP request sent, awaiting r

In [ ]:

# Function to read an article from a file and split it into sentences
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sen = []

    for sentence in article:
        # Preprocess each sentence
        sen.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sen.pop()

    return sen

In [ ]:
# Function to compute the similarity between two sentences using cosine similarity
def sentence_similarity(sen1, sen2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sen1 = [w.lower() for w in sen1]
    sen2 = [w.lower() for w in sen2]

    all_words = list(set(sen1 + sen2))

    v1 = [0] * len(all_words) # vector1
    v2 = [0] * len(all_words) # vector2

    for w in sen1:
        if w in stopwords:
            continue
        v1[all_words.index(w)] += 1

    for w in sen2:
        if w in stopwords:
            continue
        v2[all_words.index(w)] += 1

    return 1 - cosine_similarity([v1], [v2])[0][0]

In [ ]:
# Function to generate a summary of the article
def summary_generate(file_name, top_n=5, summary_length=None):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Read text and split it into sentences
    sen = read_article(file_name)

    # Generate Similarity Matrix across sentences
    sentence_similarity_matrix = np.zeros((len(sen), len(sen)))

    for i in range(len(sen)):
        for j in range(len(sen)):
            if i != j:
                sentence_similarity_matrix[i][j] = sentence_similarity(sen[i], sen[j], stop_words)

    # Rank sentences in the similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Sort the rank and pick top sentences
    ranked_sen = sorted(((scores[i], s) for i, s in enumerate(sen)), reverse=True)

    # Output the summarized text
    total_words = 0
    summary_index = 0
    while total_words < summary_length:
        total_words += len(ranked_sen[summary_index][1])
        summary_index += 1

    print("\nSummarized Text: \n", ". ".join([" ".join(ranked_sen[i][1]) for i in range(summary_index)]))
    print("\n")


In [ ]:
# Parameters:
#  "/content/msft.txt": The file path of the article/text to be summarized.
#  top_n=5: The number of top-ranked sentences to be included in the summary. Default is 5.
#  summary_length=50: The maximum number of words you want in the summarized text. The summary will include sentences until this word limit is reached.
#  If not specified (None), the summary will be based on the top_n sentences only.

summary_generate("/content/extracted_content/example/msft.txt", top_n=5, summary_length=50)



Summarized Text: 
 The program was developed to provide job ready skills to programmers who wanted to hone their skills in AI and data science with a series of online courses which featured hands-on labs and expert instructors as well. The program is an attempt to ramp up the institutional set-up and build capabilities among the educators to educate the workforce of tomorrow." The program aims to build up the cognitive skills and in-depth understanding of developing intelligent cloud connected solutions for applications across industry




In an attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills
Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services
As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses
The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning.According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, "With AI being the defining technology of our time, it is transforming lives and industry and the jobs of tomorrow will require a different skillset
This will require more collaborations and training and working with AI
That’s why it has become more critical than ever for educational institutions to integrate new cloud and AI technologies
The program is an attempt to ramp up the institutional set-up and build capabilities among the educators to educate the workforce of tomorrow." The program aims to build up the cognitive skills and in-depth understanding of developing intelligent cloud connected solutions for applications across industry
Earlier in April this year, the company announced Microsoft Professional Program In AI as a learning track open to the public
The program was developed to provide job ready skills to programmers who wanted to hone their skills in AI and data science with a series of online courses which featured hands-on labs and expert instructors as well
This program also included developer-focused AI school that provided a bunch of assets to help build AI skills.

Summarized Text:
 The program was developed to provide job ready skills to programmers who wanted to hone their skills in AI and data science with a series of online courses which featured hands-on labs and expert instructors as well. The program is an attempt to ramp up the institutional set-up and build capabilities among the educators to educate the workforce of tomorrow." The program aims to build up the cognitive skills and in-depth understanding of developing intelligent cloud connected solutions for applications across industry

In [ ]:
summary_generate("/content/extracted_content/example/computer.txt", top_n=5, summary_length=50)


Summarized Text: 
 Simple special-purpose devices like microwave ovens and remote controls are included, as are factory devices like industrial robots and computer-aided design, as well as general-purpose devices like personal computers and mobile devices like smartphones. A computer system is a nominally complete computer that includes the hardware, operating system, and peripheral equipment needed and used for full operation




A computer is a machine that can be programmed to carry out sequences of arithmetic or logical operations automatically. Modern digital electronic computers can perform generic sets of operations known as programs. These programs enable computers to perform a wide range of tasks. A computer system is a nominally complete computer that includes the hardware, operating system, and peripheral equipment needed and used for full operation. This term may also refer to a group of computers that are linked and function together, such as a computer network or computer cluster. A broad range of industrial and consumer products use computers as control systems. Simple special-purpose devices like microwave ovens and remote controls are included, as are factory devices like industrial robots and computer-aided design, as well as general-purpose devices like personal computers and mobile devices like smartphones. Computers power the Internet, which links billions of other computers and users.

Summarized Text:
 Simple special-purpose devices like microwave ovens and remote controls are included, as are factory devices like industrial robots and computer-aided design, as well as general-purpose devices like personal computers and mobile devices like smartphones. A computer system is a nominally complete computer that includes the hardware, operating system, and peripheral equipment needed and used for full operation

In [ ]:
summary_generate("/content/extracted_content/example/space.txt", top_n=5, summary_length=50)


Summarized Text: 
 A white hole is a bizarre cosmic object which is intensely bright, and from which matter gushes rather than disappears. But this is deeply controversial, because Einstein’s theory predicts the existence of a so-called singularity at the centre of black holes – a state of infinite gravity which would prevent anything from passing through to the white hole on the other side




A white hole is a bizarre cosmic object which is intensely bright, and from which matter gushes rather than disappears. In other words, it’s the exact opposite of a black hole. But unlike black holes, there’s no consensus about whether white holes exist, or how they’d be formed.They are predicted by Einstein’s theory of gravity, and are most often mentioned in the context of ‘wormholes’, in which a black hole acts as the entry point to a tunnel through space and time, ending in a white hole somewhere else in the Universe. But this is deeply controversial, because Einstein’s theory predicts the existence of a so-called singularity at the centre of black holes – a state of infinite gravity which would prevent anything from passing through to the white hole on the other side. The process starts when an old massive star collapses under its own weight and forms a black hole. But then, quantum effects occurring around the surface of the black hole halt further collapse to a singularity, and instead begin to gradually turn the black hole into a white hole that’s spewing out the original star matter again. The process is mind-bendingly slow, though, so we may be in for a very long wait to find out if white holes really exist.

Summarized Text:
 A white hole is a bizarre cosmic object which is intensely bright, and from which matter gushes rather than disappears. But this is deeply controversial, because Einstein’s theory predicts the existence of a so-called singularity at the centre of black holes – a state of infinite gravity which would prevent anything from passing through to the white hole on the other side


References

https://medium.com/analytics-vidhyatext-summarization-using-nlp-3e85ad0c6349

https://github.com/edubey/text-summarizer/blob/master/msft.txt

https://www.analyticsvidhya.com/blog/2020/12tired-of-reading-long-articles-text-summarization-will-make-your-task-easier/

https://medium.com/analytics-vidhya/text-summarization-in-python-using-extractive-method-including-end-to-end-implementation-2688b3fd1c8c

